In [2]:
import random
import numpy as np

In [3]:
population = 2
mutations = 0.25

In [4]:

colors = [0,1,2,3,4,5]

board = random.choices(colors,k=4)
print(board)

[1, 0, 2, 2]


In [5]:
def ga(array, population, mutations):

    score_memoization = {}

    def score(board, choice):
        
        key = tuple(choice)
        
        # if key in score_memoization:
        #     return score_memoization[key]

        tmp_board = board.copy()

        in_place = 0
        in_place_list = set()
        same_color = 0

        for i in range(len(choice)):
            if choice[i] == board[i]:
                in_place += 1
                in_place_list.add(i)
                tmp_board[i] = -1

        for i,c in enumerate(choice):
            if i not in in_place_list and c in tmp_board:
                same_color += 1

        #score = (2 * in_place) + same_color
        score = in_place
        score_memoization[key] = score

        return score

    mid = 4//2

    mem = np.random.randint(6, size=(2 * population, 4))
    scores = np.zeros((2 * population))
    bottom = list(range(len(mem)))

    for i in range(1000000):
        
        # Bottom will contain all the random individuals generated when starting the execution
        # and the new individuals after the first iteration. Bottom means the bottom of the list
        # sorted by score
        for k in bottom:
            scores[k] = score(array, mem[k])

        # Check if the solution has been found
        max_score = np.argmax(scores)
        if scores[max_score] == 4:
            return i, len(score_memoization)

        # Select the population of individuals according to the score function
        top_n_scores = np.argpartition(scores, population)
        top = top_n_scores[population:]
        bottom = top_n_scores[:population]

        # Create #population new elements from the crossover and mutation
        for j in range(population):

            # Crossover -> Select parents from the top individuals
            #
            # I tried this with random choice and just picking a random position
            # from the top and the next one and the result is the same but way faster
            # It might be because of either the randomization of the initial population or maybe
            # the implementation of argpartition? or both?
            
            r = random.randrange(len(top))  
            idx = [r, (r+1)%len(top)]
            parents = [top[idx[0]],top[idx[1]]]
            
            mem[bottom[j]][0:mid] = mem[parents[0]][0:mid]
            mem[bottom[j]][-(mid):] = mem[parents[1]][-(mid):]

            # Mutation -> Mutate the bits
            #
            # The random choice of the bits to mutate is the most costly of the implementation
            # It seems there has to be some way to speed up this 
            idx = np.random.choice([0,1,2,3,4,5,6], p=[(1-mutations), mutations/6,mutations/6,mutations/6,mutations/6,mutations/6,mutations/6],size=(4))

            for k in range(len(idx)):
                mem[bottom[j]][k] = (mem[bottom[j]][k] + idx[k])%6

In [8]:
trials = 1000

accum1 = 0
accum2 = 0

for i in range(trials):
    iters, mem = ga(board, population, mutations)
    accum1 += iters
    accum2 += mem

print (accum1/trials)
print (accum2/trials)

34.828
36.11
